### 13.
Read about Broadcasting with Arrays on the chapter Computation on Arrays:
Broadcasting from Python Data Science Handbook (J. VandePlas, 2016):
Link: https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computationon-arrays-broadcasting.html.

### 14.
Read Rewriting the particle simulator in Numpy on Chapter 2: Fast Array Operations
with Numpy and Pandas (pp. 68) from the book G. Lenaro (2017). Python high
Performance. Second Edition. UK: Packt Publishing Ltd. Implement the
improvements on the particle simulator using NumPy. Show that both
implementations scale linearly with particle size, but the runtime in the pure Python
version grows much faster than the NumPy version. 

In [7]:
import numpy as np
from random import uniform

class Particle:
    def __init__(self, x, y, velocidad_angular):
        self.x = x
        self.y = y
        self.velocidad_angular = velocidad_angular

class ParticleSimulator:

    def __init__(self, particles):
        self.particles = particles

    def evolve_python(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        for i in range(nsteps):
            for p in self.particles:
                norm = (p.x**2 + p.y**2)**0.5
                v_x = (-p.y) / norm
                v_y = p.x / norm
                d_x = timestep * p.velocidad_angular * v_x
                d_y = timestep * p.velocidad_angular * v_y
                p.x += d_x
                p.y += d_y

    def evolve_numpy(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        r_i = np.array([[p.x, p.y] for p in self.particles])
        velocidad_angular_i = np.array([p.velocidad_angular for p in self.particles])
        for i in range(nsteps):
            norm_i = np.sqrt((r_i ** 2).sum(axis=1))
            v_i = r_i[:, [1, 0]] * np.array([-1, 1]) / norm_i[:, np.newaxis]
            d_i = timestep * velocidad_angular_i[:, np.newaxis] * v_i
            r_i += d_i
        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]

 
def benchmark(npart=1000, method='python'):
    particles = [Particle(uniform(-1.0, 1.0), uniform(-1.0, 1.0), uniform(-1.0, 1.0)) for _ in range(npart)]
    simulator = ParticleSimulator(particles)
    if method == 'python':
        simulator.evolve_python(0.1)
    elif method == 'numpy':
        simulator.evolve_numpy(0.1)

if __name__ == "__main__":
    from timeit import timeit
    print("Only python")
    print(timeit("benchmark(10000, 'python')", globals=globals(), number=1))
    print("With numpy")
    print(timeit("benchmark(10000, 'numpy')", globals=globals(), number=1))

Only python
46.77692069999989
With numpy
1.8440700999999535


###  15. 
Explain how to optain the optimal performance with numexpr. Read the section
Reaching optimal performance with numexpr, pp. 72 from the previous reference.
Implement it and measure the execution time.

In [8]:
import numpy as np
import numexpr as ne

# Generate a large array
A = np.random.rand(10000000)

In [9]:
%%timeit

# Perform the operation using NumPy
B_numpy = np.sin(A)**2 + np.cos(A)**2

253 ms ± 21.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
# Perform the operation using NumExpr
expression = "sin(A)**2 + cos(A)**2"
B_numexpr = ne.evaluate(expression)


35.7 ms ± 1.63 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
